In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
from dmsbatch import create_batch_client, create_blob_client
import datetime

## First create a batch client from the config file

In [ ]:
client = create_batch_client('../tests/data/dmsbatch.config')
blob_client = create_blob_client('../tests/data/dmsbatch.config')

## Application packages
To copy large files and programs it is best to zip (or targz) them and upload them as application packages

Application packages are setup separately in either azure management apis or from the web console or cli tool

These are referenced here by their name and version
e.g. DSM2, python and other programs

One extra field (last one) is the path within the zip file where the executables can be found. These are used later to setup the PATH varible

In [ ]:
app_pkgs = [('dsm2linux', '8.2.8449db2', 'DSM2-8.2.8449db2-Linux/bin')]

### Show vms available

https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series

In [ ]:
#display(client.skus_available())

### Create or resize existing pool
If the pool doesn't exist it will create it
If the pool exists, it will resize to the second arg

In [ ]:
client.set_path_to_apps(app_pkgs,ostype='linux')

In [ ]:
pool_start_cmds = ['printenv',
'yum install -y glibc.i686 libstdc++.i686 glibc.x86_64 libstdc++.x86_64',# --setopt=protected_multilib=false',
'yum-config-manager --add-repo https://yum.repos.intel.com/2019/setup/intel-psxe-runtime-2019.repo',
'rpm --import https://yum.repos.intel.com/2019/setup/RPM-GPG-KEY-intel-psxe-runtime-2019',
'yum install -y intel-icc-runtime-32bit intel-ifort-runtime-32bit']
client.wrap_commands_in_shell(pool_start_cmds,ostype='linux')

In [ ]:
created_pool = client.create_pool('dsm2linuxpool',
                   1,
                   app_packages=[(app,version) for app,version,_ in app_pkgs], 
                   vm_size='standard_f2s_v2', 
                   tasks_per_vm=2,
                   os_image_data=('openlogic', 'centos', '7_8'),
                    start_task_cmd=client.wrap_commands_in_shell(pool_start_cmds,ostype='linux'),
                    start_task_admin=True,
                    elevation_level='admin'
                  )

In [ ]:
if not created_pool:
    client.resize_pool('dsm2linuxpool',1)

### Create job on pool or fail if it exists
Jobs are containers of tasks (things that run on nodes (machines) in the pool). If this exists, the next line will fail

In [ ]:
try:
    client.create_job('dsm2linuxjobs','dsm2linuxpool')
except Exception as exc:
    print('Job already exists? Its ok, we will continue')

### Create a task
This uses the application package as pre -set up. If not, create one https://docs.microsoft.com/en-us/azure/batch/batch-application-packages

In [ ]:
tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
task_name = f'hydro_version_{tsnow}'
cmd_string = client.wrap_cmd_with_app_path(
    """
source /opt/intel/psxe_runtime/linux/bin/compilervars.sh ia32;
%s;
hydro -v; 
unzip -v;
"""%(client.set_path_to_apps(app_pkgs,ostype='linux')),app_pkgs,ostype='linux')
print(task_name)
print(cmd_string)

In [ ]:
blob_client.blob_client.account_name

In [ ]:
import dmsbatch
permissions = dmsbatch.commands.BlobSasPermissions(write=True)
# |helpers.azureblob.BlobPermissions.ADD|helpers.azureblob.BlobPermissions.CREATE
output_dir_sas_url = blob_client.get_container_sas_url(
                                                   'dsm2linuxjobs',
                                                   permissions)
print(output_dir_sas_url)

In [ ]:
std_out_files = client.create_output_file_spec(
    '../std*.txt', output_dir_sas_url, blob_path=f'{task_name}')

In [ ]:
hydro_v_task = client.create_task(task_name,cmd_string,output_files=[std_out_files])

### Next submit the task and wait 

In [ ]:
client.submit_tasks('dsm2linuxjobs',[hydro_v_task])

In [ ]:
client.wait_for_tasks_to_complete('dsm2linuxjobs', datetime.timedelta(seconds=100))

## Finally resize the pool to 0 to save costs

In [ ]:
client.resize_pool('dsm2linuxpool',0)